In [ ]:
import asyncio
import math
import numpy
import astropy.coordinates
import astropy.time
import astropy.units as u

from lvmtipo.site import Site
from lvmtipo.siderostat import Siderostat
from lvmtipo.fiber import Fiber
from lvmtipo.target import Target
from lvmtipo.actors import lvm

device_module = 2
derot_buffer = 100

await lvm.sci.km.start()
await lvm.sci.pwi.start()

print(await lvm.sci.km.status())
print(await lvm.sci.pwi.status())


In [17]:

delta_time=2.0
azang = 180.0
medSign = 1
sid = Siderostat(azang=azang, medSign=medSign)

homeOffset = 135.0
homeIsWest = False
geoloc = Site(name = "MPIA")

ra_h, dec_d = (await lvm.sci.pwi.status()).flatten().unpack("ra_j2000_hours", "dec_j2000_degs")

targ = astropy.coordinates.SkyCoord(ra=ra_h, dec=dec_d, unit=(u.hourangle, u.deg))
target = Target(targ)

delta_time=10
polynoms=6

now = astropy.time.Time.now()
traj = sid.mpiaMocon(geoloc, 
              target,
              None,
              deltaTime=delta_time,
              homeIsWest=homeIsWest,
              homeOffset=homeOffset,
              polyN=polynoms,
              time=now)

print(targ)
print(traj)

<SkyCoord (ICRS): (ra, dec) in deg
    (0., 0.)>
[[16276, 3120, 2682892, 0, 0], [16276, 3122, 2683667, 0, 0], [16276, 3123, 2684442, 0, 0], [16276, 3124, 2685218, 0, 0], [16276, 3126, 2685994, 0, 0], [16276, 3127, 2686770, 0, 0], [0, 0, 2687547, 0, 0]]


In [30]:

await lvm.sci.km.moveAbsolute(traj[0][2], "STEPS")

# clear buffer
try:
   print(await lvm.sci.km.chat(1, 226, device_module))

except Exception as ex:
   pass

# create buffer
print (await lvm.sci.km.chat(1, 220, device_module, derot_buffer))

# upload trajectory
for i, t in enumerate(traj):
   print (await lvm.sci.km.chat(1, 221, device_module, 0, f"{i} {t[0]} {t[1]} {t[2]} {t[3]} {t[4]}"))


# start profile
print(await lvm.sci.km.chat(1, 222, device_module, 0))

await asyncio.sleep(2.0)
print(await lvm.sci.km.chat(1, 225, device_module))


  

{'ChatRc': ['1 226 2 1']}
{'ChatRc': ['1 220 2 1']}
{'ChatRc': ['1 221 2 1']}
{'ChatRc': ['1 221 2 1']}
{'ChatRc': ['1 221 2 1']}
{'ChatRc': ['1 221 2 1']}
{'ChatRc': ['1 221 2 1']}
{'ChatRc': ['1 221 2 1']}
{'ChatRc': ['1 221 2 1']}
{'ChatRc': ['1 222 2 1']}
{'ChatRc': ['1 225 2 2 1', '1 225 2 1']}


In [31]:
moidx = 0
while moidx < len(traj):
    try:
       rc = (await lvm.sci.km.chat(1, 225, device_module)).unpack()
       moidx = int(rc[0].split(' ')[-1])
       print(moidx)
       await asyncio.sleep(delta_time)

    except Exception as ex:
       print(ex)


2
3
4
5
6
7


In [27]:
print(await lvm.sci.km.chat(1, 224, device_module))


{'ChatRc': ['1 224 2 1']}
